In [1]:
import os
os.chdir('/mnt/jw01-aruk-home01/projects/psa_functional_genomics/RA_challenge/RA2_alpine_lads')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras 
import os
import tensorflow as tf
import tensorflow_addons as tfa
from utils.config import Config
import dataset.train_dataset as dtd
import PIL
import PIL.ImageOps
from tensorflow.keras.utils import plot_model
test_config = Config()
from tensorflow.keras.models import load_model
from model import NASnet_multioutput
import itertools
import seaborn as sns

In [3]:
from dataset.train_dataset import train_dataset

dataset = train_dataset(test_config)

hands_dataset, feet_dataset, hands_dataset_val, feet_dataset_val = dataset.initialize_pipeline()

WARNING - Missing permissions to create directory for caching!
WARNING - Missing permissions to create directory for caching!


In [29]:
def get_output(x,y):
    return x,tf.split(y,[12,1],1)[1]
feet_dataset1 = feet_dataset.map(get_output)
feet_dataset_val1 = feet_dataset_val.map(get_output)


In [30]:
model = NASnet_multioutput.create_NASnet_multioutupt(test_config)
model.load_weights("weights/NIH_chest_NASnet_model_275_step2")

In [31]:
new_model = keras.models.Sequential()
new_model.add(model.layers[0])
new_model.add(model.layers[1])
new_model.add(model.layers[2])
new_model.add(keras.layers.Dense(1,activation="linear",name="State"))
for layer in new_model.layers[:-1]:
    layer.trainable = False
new_model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mae"])

In [32]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 16, 12, 1056)      4269140   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1056)              0         
_________________________________________________________________
State (Dense)                (None, 1)                 1057      
Total params: 4,270,197
Trainable params: 1,057
Non-trainable params: 4,269,140
_________________________________________________________________


In [33]:
history = new_model.fit(
    feet_dataset1,validation_data=feet_dataset_val1,epochs=10, steps_per_epoch=50,
    validation_steps=2
)

Train for 50 steps, validate for 2 steps
Epoch 1/10
50/50 [==============================] - 84s 2s/step - loss: 46.7077 - mae: 3.3764 - val_loss: 98.2396 - val_mae: 4.7712
Epoch 2/10
50/50 [==============================] - 71s 1s/step - loss: 37.9883 - mae: 3.5845 - val_loss: 101.2489 - val_mae: 4.6429
Epoch 3/10
50/50 [==============================] - 71s 1s/step - loss: 42.2843 - mae: 3.6337 - val_loss: 103.0572 - val_mae: 4.5984
Epoch 4/10
50/50 [==============================] - 71s 1s/step - loss: 45.3957 - mae: 3.7322 - val_loss: 103.1957 - val_mae: 4.6040
Epoch 5/10
50/50 [==============================] - 70s 1s/step - loss: 41.5993 - mae: 3.6009 - val_loss: 104.6215 - val_mae: 4.5815
Epoch 6/10
50/50 [==============================] - 71s 1s/step - loss: 39.4548 - mae: 3.6547 - val_loss: 107.0642 - val_mae: 4.5777
Epoch 7/10
50/50 [==============================] - 71s 1s/step - loss: 44.6903 - mae: 3.5921 - val_loss: 109.1576 - val_mae: 4.5965
Epoch 8/10
50/50 [===========

In [ ]:
# unfreeze

In [35]:
for layer in new_model.layers[:-1]:
    layer.trainable = True
new_model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mae"])

In [36]:
history = new_model.fit(
    feet_dataset1,validation_data=feet_dataset_val1,epochs=10, steps_per_epoch=50,
    validation_steps=2
)

Train for 50 steps, validate for 2 steps
Epoch 1/10
50/50 [==============================] - 250s 5s/step - loss: 40.6125 - mae: 3.4866 - val_loss: 119.3402 - val_mae: 5.0274
Epoch 2/10
50/50 [==============================] - 226s 5s/step - loss: 30.1457 - mae: 3.0581 - val_loss: 153.1487 - val_mae: 7.6116
Epoch 3/10
27/50 [===============>..............] - ETA: 1:44 - loss: 18.4667 - mae: 2.5578

KeyboardInterrupt: 

In [37]:
# test
for image, label in feet_dataset_val1.take(5):
    N = image.shape[0]
    D = label.shape[1]
    
    E = np.zeros((N, D))

    for n in range(N):
        y_pred = new_model.predict(np.array(image[n]).reshape(1,test_config.img_height,test_config.img_width,1))
        y = label[n]
        
        
        E[n, :] = np.square(y_pred - y)
        
    rmse = np.sqrt(np.mean(E, axis = 0))
        
    print("Batch RMSE: ", rmse)

Batch RMSE:  [13.20908778]
Batch RMSE:  [12.60952021]
Batch RMSE:  [9.48683298]
Batch RMSE:  [15.60384568]
Batch RMSE:  [12.79531164]
